# Revive Analysis

# Dataset

Let's first load the dataset

In [ ]:
%pip install -q pandas openpyxl

In [1]:
import pandas as pd
from os import listdir

In [2]:
DATASET_FOLDER = "./data/my_revives/"

In [3]:
listdir(DATASET_FOLDER)

['torn-revives-2025-10-22.xlsx']